<font size=5>**Movie Recommender - The Models Combine (2020)**</font>

***This notebook is an individual project submitted as a written assessment by Timofey Sadovnikov***

<font size=4>**Assessment Course Goals**</font>

<font size=3>S7 - Communicate problems, recommendations and insights adapted to the intended target audience</font>

<font size=3>S10 - Plan, execute and identify resources for carrying out experiments to draw data-informed conclusions</font>

<font size=3>C1 - Process and analyze data in such a way that it leads to further learning and professional development</font>

<font size=3>C3 - Independently be able to work methodically and flexibly in various projects and processes</font>


# Project Background (S7)

Amazon, Google, Facebook, Netflix and the list of prominent tech companies could go on, they all invest extensively in developing machine learning models for various purposes like recommending relevant content to their users to increase retention and revenue. By researching the fundamentals of how different models work and the python skills required to develop these I set out to create a recommendation system of my own. 

Through the search of fitting datasets I found that using movie data fit my goal. As I could create a content based recommender where you recommend movies by the feature similarities between movies, eg. genre, actors, directors and other features of a movie. As well as a recommender using collabrative filtering (taking user-ratings into account).

However, there are plenty projects on github and kaggle where this has already been done. With that said I wanted to make sure to stop myself from just copying others' code whenever a problem arises as I wanted to use this opportunity to learn and create a project I could own. Therefore I set a challenge to create a hybrid recommender, where I combine these two models together, which even though is definitely the case of how recommendation systems function in the industry. I've yet to find public projects of this, at least with the use of my data sources.

# Loading Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Loading Data

# **First look and overview of the data** (C1)
<font size="4"> **IMDb Dataset**</font>

The dataset I used for the content based system is a smaller subset of the public IMDb dataset, as it hosts vast amount of features that can uniquely describe any movie which is important for this model to find distinct similarities between movies.


In [ ]:
#IMDb dataset
imdb = pd.read_csv('../input/imdb-extensive-dataset/IMDb movies.csv')

#MovieLens dataset
ratings = pd.read_csv('../input/movielens-latest-small/ratings.csv')
movies = pd.read_csv('../input/movielens-latest-small/movies.csv')
id_links = pd.read_csv('../input/movielens-latest-small/links.csv')

In [ ]:
imdb.columns

In [ ]:
imdb.shape

In [ ]:
imdb.head()

<font size="4"> **MovieLens data**</font>

This dataset will be used to create the recommendation system that uses collabrative filtering, as the data contains unique user ratings, movies and the "links" dataset that hosts the imdb id of those movies.

In [ ]:
movies.columns, ratings.columns, id_links.columns

In [ ]:
movies.shape, ratings.shape, id_links.shape

In [ ]:
movies.head() 

In [ ]:
ratings.head()

In [ ]:
id_links.head()

<font size="4">Dataset compability takeaways</font>
* Differing imdb-id format in **imdb** and **id_links**. imdb: "ttxxxxxxxxx" (x=digits) id_links: "xxx" digits with no leading zeros
* Largely differing amount of movies in the between the movielens and imdb datasets
* The format of the titles differ in the datasets as movielens titles have a trailing movie release year


# IMDb dataset preprocessing - Content Based Model (S7, S10, C1, C3)

<font size="4">Steps:</font>
* Check for null values and fill them
* Get keywords by removing all stopwords from movies' "description"
* Create new dataframe by extracting movies and valuable features for the content based system
* Transform the id in IMDb dataset to digits with no leading zeros to match id_links imdbid
* Format all feature values for the creation of "Bag of Words"
* Create bag of words from feature values
* Convert bag of words into a sparse matrix representating the token counts
* Find cosine similarity between the movies in the sparse matrix






In [ ]:
imdb.columns

In [ ]:
#Picking all movie features that I reckon to be most valuable for the recommendation system
df = imdb[['imdb_title_id','title', 'year', 'genre', 'language', 'director', 'writer', 'actors', 'description']]

**Disclaimer:**
To not overfit the algorithm with too much data, I will decide on fewer features later that matches my recommendation purpose more appropriately.

In [ ]:
#Check for null values
df.isnull().sum()

In [ ]:
#fill null values with whitespace as words like "NaN" or "Missing Values" can increase similarity between movies with missing values
df = df.fillna(' ')

In [ ]:
#Getting keywords from description by filtering stopwords from the nltk library
stop = set(stopwords.words('english'))
df['keywords'] = df['description'].str.split().apply(lambda x: [item for item in x if item not in stop])
df['keywords'] = df['keywords'].str.join(' ')
df.drop('description', axis=1, inplace=True)

In [ ]:
#Formatting dataframe and its' values for bag of words
df['imdb_title_id'] = df['imdb_title_id'].str.extract(r'([1-9][0-9]*|0)\b', expand=False)
df = df.rename(columns={'imdb_title_id':'movie_id'})
df['genre'] = df['genre'].str.replace(',', '')
df['actors'] = df['actors'].str.replace(',', '')
df['writer'] = df['writer'].str.replace(',', '')
df['director'] = df['director'].str.replace(',', '')
df['keywords'] = df['keywords'].str.lower()
df = df.astype(str)
df.head()

In [ ]:
#create function that combines features into bag of words

# def combine_features(row):
#     return row['director'] + " " + row['genre'] + " "+ row['year'] + " " + row['keywords'] + " " + row['actors'] + " " + row['language'] + " " + row['writer']

def combine_features(row):
    return row['director'] + " " + row['genre'] + " " + row['actors'] + " " + row['keywords']

#combine features into bag of words
df['bow'] = df.apply(combine_features, axis=1)
df.head()

In [ ]:
#Using a smaller sample of dataset for the model incase of memory- and runtime errors
df1 = df.head(10000)

# The algorithm - Cosine Similarity (S7, S10, C1, C3)
For this model, I'll use the cosine similarity to define the "similarity" between the movies.
Now similar to finding **euclidean distance** between two points (the distance between the vector endpoints) through the usage of pythagoras thereom, you continue to calculate the angle between the vectors and the cosine similarity cosine(angle between the vectors).

The reason for not just using just the distance between the vectors as a similarity score even though it might seem reasonable, the lengths of vectors can affect the euclidian distance but not cosine distance **(1 - cos(angle))**. As the lengths of two vectors defines the size of it (eg. the amount of data), but the angle better defines their 
characteristics(?) in lack of better words. Therefore the angular distance is better suited to define their similarity.

<font size="4">**I help illustrate the problem through Paint below:**</font>
![](https://i.imgur.com/OscO5cf.png)
As you can see the euclidian distance becomes very much longer on the second graph even though the angle is fairly small

So basically if the angle between two vectors is 45 degrees then cosine similarity = Cos(45)=0.53 or 90 degrees cos(90) = 0 or if the vectors have opposite directions cos(180) = -1 etc etc.

<font size="4">**Convert bag of words into token counts**</font>

To actually be able to calculate the cosine similarity between movies' bag of words, we need reform our data into something that can be mathematically operated.

Therefore we fit our bag of words into has a function called CountVectorizer() from the sklearn library which converts our data into a sparse matrix where the words are tokenized and counted. (the index order of the dataframe will be retained in the matrix)

In [ ]:
#convert bag of words into a matrix of token counts
cv = CountVectorizer()
count_matrix = cv.fit_transform(df1['bow'])

<font size="4">**Find cosine similarity**</font>

And to our lazy-coder-comfort, the same library has a function cosine_similarity that takes a sparse matrix as input and returns the cosine similarity between the vectors.

In [ ]:
#Find cosine similarity between the movies in the sparse matrix
cos_sim = cosine_similarity(count_matrix)

# Content Based Recommender - IMDb Dataset (S7, S10, C1, C3)
First we create a function that takes a string movie name and integer/float as rating. Rating will be used to determine if you liked or disliked movie, in case of the movie having a rating of less than 2.5, then you won't recommend movies similar to it. Higher the rating, the more value is given to the similar movies which will be useful further on when we pass a list of movies with ratings.

In [ ]:
#DataFrame with indices to retrieve movie titles and other features
features_imdb = df1[['movie_id','title','year']].reset_index()


def recommend_sim_features(movie, rating):
    
    movie_index = features_imdb[features_imdb['title'] == movie]['index'].values[0] #get index from movie title to retrieve similarity scores from the similarity matrix
    
    sim_scores = pd.Series(cos_sim[movie_index]).sort_values(ascending=False) #find the highest similarity scores
    
    sim_feat_movies = pd.DataFrame({ 'score' : sim_scores[1:101]*(rating-2.5)}).reset_index() #Create dataframe with the movies' 100 highest scored movies (itself excluded)
    sim_feat_movies = sim_feat_movies.merge(features_imdb) #merge with features_imdb to retrieve movie_id and year
    sim_feat_movies = sim_feat_movies[['movie_id','title', 'year', 'score']] #change order of columns and drop index
    
    return sim_feat_movies


<font size=4> **Testing Function** </font>

In [ ]:
#Printing 10 random titles from dataframe to choose from and test in our function
df1.iloc[np.random.choice(np.arange(len(df1['year'].astype(int) > 2000)), 10, False)]['title']

In [ ]:
recommend_sim_features('The Rocking Horse Winner', 5).head()

<font size=4>**Recommending from a list of movies - Content Based**</font>

Now most often we have a longer watch history than just one movie which summarizes our prefrences better, therefore next up we create a function that loops through a list of movies and their respective rating

In [ ]:
#Return a dataframe with top 'n' recommended movies from a list of tuples with movies and their respective ratings.
#Example: movies_watched = [(Sharknado, 5), (Sharknado 2, 5), (Pulp Fiction, 1)]

def movie_recommender_cb(user_movies, n=100):
    
    recommended_movies = pd.DataFrame() #create dataframe to append similar movies
    
    #Loop through list one by one and append every returned dataframe to recommended_movies
    for movie, rating in user_movies:
        #Error management in case of a movie in the list can't be found in the dataset
        try:
            recommended_movies = recommended_movies.append(recommend_sim_features(movie, rating))
        except: print(movie + ' was not in the IMDb dataset')
            
    #Group movies and sum the score, sort by highest score descending
    recommended_movies = recommended_movies.groupby(['movie_id', 'title', 'year']).sum().sort_values('score', ascending=False).reset_index()
    
    return recommended_movies.head(n) #return n (default=100) most similar movies

In [ ]:
#Random test list of movie ratings
test_list_imdb = [('High Society', 4), ("Paris Playboys", 3), ('Here Come the Marines', 4)]
#return top 5 highest scored movies
movie_recommender_cb(test_list_imdb, 5)

# MovieLens dataset preprocessing - Collabrative Filtering Model (S7, S10, C1, C3)

<font size="4">Steps:</font>
* Extracting the year and removing it from title to match the title-format in the IMDb dataset
* Merge datasets, set the imdb-id as the unique identifier
* Rename columns to match snake case naming style
* Drop unnessecary columns
* Convert dataframe to a matrix with values representing all users' ratings of all movies
* Find similarity by calculating correlation between movies based on users' ratings. 




In [ ]:
#Using regex pattern to extract year from title and removing year from title.
movies['year'] = movies['title'].str.extract('(\d{4})', expand = False)
movies['title'] = movies['title'].str.replace('(\s.\d{4}.)', '')

In [ ]:
#Combine datasets to one dataframe
df2 = pd.merge(ratings, movies)
df2 = df2.merge(id_links)
df2.head()

In [ ]:
df2 = df2.rename(columns={'imdbId':'movie_id', 'userId':'user_id'})
df2.drop(['movieId','timestamp','genres','tmdbId'], axis=1, inplace=True)


# The algorithm - Pearson's correlation coefficient (S10)

**Pearsons correlation algorithm** or **"Pearson's r"** calculates the strength of the linear relationship between datapoints.
Pearson's r is between 1 and -1, where 1 is the strongest possible positive relationship (where as x increases, so does y), -1 is strongest possible negative relationship (where as x increases, y decreases) and whereas 0 means there is no relationship between the datapoints. 

We will use Pearson's r as a measure to determine similarity between movies.

# Collabrative filtering approach (S7, S10, C1, C3)

Before we apply any algorithm to our data we need to decide in what way we want to recommend. As from our data we can use collabrative filtering in two different ways: **User to User** or **Movie to Movie**

<font size="4">**I've tried to illustrate and explain the difference below:**</font>
![](https://i.imgur.com/1Otiu7J.png)
Depending on the approach we choose, we want the data to be formatted accordingly before we apply the algorithm to calculate their correlation.
You might notice that the recommendation outcome in the picture for the two methods is the same, User 3 gets recommended Movie 5 in either case...
However, when it comes to a large dataset this could look different. 

<font size="4">So which is better?</font>

**User-Based CF** works better when there are a many users with a high rating percentage, because then the recommender can connect between different users with a high similarity. However this wouldn't work well for recommending to new users who haven't yet rated a sufficient amount of movies to distinctify their taste into a group of similar users.

**Movie-Based CF** works better when the latter is not the case, if we have a few amount of users we would be able to say which users are MOST similar, but that similarity will most probably not actually reflect reality as we don't have a large enough population of users to reach statistical significance. Supposing this is the case it's better to use Movie-Based CF by attributing the ratings to the movies.

So let's inspect our dataset and see which fits our dataset more.

In [ ]:
#count of unique users
x_1 = df2['user_id'].nunique()
#count of unique movies
x_2 = df2['movie_id'].nunique()
#count of ratings
x_3 = len(df2)
#Average rating count by movie
x_4 = df2['rating'].count()  / df2['user_id'].nunique()
#Average rating count by movie
x_5 = df2['rating'].count()  / df2['movie_id'].nunique()

print(f'Count of unique users: {x_1} \nCount of unique movies: {x_2} \nCount of ratings: {x_3} \nAverage rating count by users: {x_4} \nAverage rating count by movie: {x_5}')

<font size=4>Conclusions:</font>

<font size=3>We can see we have a small amount of users but with a lot of ratings amounted to them.
This would work good if we were trying to introduce unseen movies to those exact users based on their ratings.</font>

<font size=3>In our case, we will be recommending new movies to a cold start user by fitting a small list of liked and/or disliked movies into our model. Therefore a movie-based CF should in theory work better. (Unless we are to pass a very large list of movies, which would be a hastle to anyone)
</font>

In [ ]:
#Pivoting our dataframe to match the format of a Movie-Based CF
user_ratings = df2.pivot_table(index=['user_id'], columns=['title'], values='rating')
user_ratings.head()

In [ ]:
#As we saw the mean rating count by movie was 10 which was quite small, so let's keep that as a minimum instead 
user_ratings = user_ratings.dropna(thresh=10 ,axis=1).fillna(0, axis=1) # fill NaN values and drop movies with ratings count less than thresh(10)
user_ratings.head()

In [ ]:
#Apply Pearsons Correlation to find movie similarity based on users' ratings
movie_similarity = user_ratings.corr(method='pearson')
movie_similarity.head()

# Movie-Based CF Recommender - MovieLens dataset (S7, S10, C1, C3) 

<font size=3>We create a function that takes movie and rating as input, and just like the CB-Recommender: </font>
> Rating will be used to determine if you liked or disliked movie, in case of the movie having a rating of less than 2.5, then you won't recommend movies similar to it. Higher the rating, the more value is given to the similar movies which will be useful further on when we pass a list of movies with ratings.

In [ ]:
#Function that retrieves the movies' correlation to other movies and scoring it based on the rating.
def get_sim_ratings(movie, rating):
    sim_ratings = movie_similarity[movie]*(rating-2.5)
    sim_ratings = sim_ratings.sort_values(ascending=False)
    return sim_ratings


<font size=3>**As we are later going to merge our CB Recommending system and the CF Recommending system, let's transform the output to a format indentical of the CB-Recommender.**

**So let's compare the formats as of now:**</font>

In [ ]:
movie_recommender_cb(test_list_imdb, 2)
#returns dataframe with movie_id, title, year and score

In [ ]:
get_sim_ratings('Toy Story', 5)[:10]
#Returns series object with title and respective score

<font size=3>**The output returns the movie itself as the most similar (obviously) and we also have to convert the series output from "get_sim_ratings()" into a dataframe and retrieve movie_id and year.**</font>

In [ ]:
#DataFrame to retrieve relevant movie features
features_movielens = df2[['movie_id','title','year']].drop_duplicates()

def recommend_sim_ratings(movie, rating):
    
    sim_ratings = get_sim_ratings(movie, rating).to_frame().reset_index() #get the series output and convert to dataframe using to_frame() and extract titles column from index
    sim_ratings.rename(columns={movie:'score'}, inplace=True) #naming the column with score values to 'score' as it's otherwise set to be the movie title
    
    sim_ratings = sim_ratings.merge(features_movielens) #merge to retrieve movie_id and year
    sim_ratings = sim_ratings[['movie_id', 'title', 'year', 'score']] #change order of columns to match the ouput order of movie_recommender_cb()
    sim_ratings = sim_ratings.sort_values('score', ascending=False)[1:101] #return
    return sim_ratings

#testing with top 3 most similar movies
recommend_sim_ratings('Toy Story', 5)[:3]

<font size=4> **Recommending from a list of movies - Movie-Based CF**</font>

<font size=3>**As the output format is now identical to the to the CB-Recommender, we can use an almost identical code to iterate through the list of movies.**</font>


In [ ]:
def movie_recommender_cf(user_movies, n=100):
    
    recommended_movies = pd.DataFrame() #create dataframe to append similar movies
    
    #Loop through list one by one and append every returned dataframe to recommended_movies
    for movie, rating in user_movies:
        #Error management in case of a movie in the list can't be found in the dataset
        try:
            recommended_movies = recommended_movies.append(recommend_sim_ratings(movie, rating))
        except: print(movie + ' was not in the MovieLens dataset')
            
    #Group movies and sum the score, sort by highest score descending
    recommended_movies = recommended_movies.groupby(['movie_id', 'title', 'year']).sum().sort_values('score', ascending=False).reset_index()
    
    return recommended_movies.head(n) #return n most similar movies

In [ ]:
#Printing 10 random titles from dataframe to choose from and test in our function
df2.iloc[np.random.choice(np.arange(len(df2)), 10, False)]['title']

In [ ]:
test_list_movielens = [('Toy Story', 5), ('Back to the Future', 4), ('Harry Potter and the Deathly Hallows: Part 2', 4)]

#return top 5 most highest scored movies
movie_recommender_cf(test_list_movielens, 5)

# Hybrid Movie Recommender - CB + CF (S10, C1)

<font size=3>Finally as we have made our recommenders return identically formatted outputs, we don't have to do much to create our hybrid system. Infact all we really have to do is **merge, group, sum and sort** the output of our two recommenders. However to make it a bit more sophisticated we can add a scoring factor, to regulate the impact a model will have on the hybrid recommender</font>


In [ ]:
def hybrid_movie_recommender(user_movies, n=10, score_factor=0.5):
    if score_factor < 1 and score_factor > 0:
        print(f'Content Based impact: {round(score_factor*100)}% \nMovie-Based CF impact: {round((1-score_factor)*100)}%')
        cb = movie_recommender_cb(user_movies)
        cb['score'] = cb['score']*(1 + score_factor)
        cf = movie_recommender_cf(user_movies)
        cf['score'] = cf['score']*(1 + (1 - score_factor))
    else: 
        print('Score factor was set to default 0.5, must be between 0 and 1')
        score_factor=0.5
        cb = movie_recommender_cb(user_movies)
        cb['score'] = cb['score']*(1 + score_factor)
        cf = movie_recommender_cf(user_movies)
        cf['score'] = cf['score']*(1 + (1 - score_factor))
    
    recommended_movies = cf.append(cb).groupby(['movie_id', 'title', 'year']).sum().sort_values('score', ascending=False).reset_index() #merge, group, sum and sort

    return recommended_movies.head(n) #return n most similar movies (n default=10)


hybrid_movie_recommender((test_list_movielens + test_list_imdb), 10, 0.55) # testing hybrid recommender with score factor of 0.55

# Disclaimer: (S7)

<font size=3>There was no other purpose in creating this hybrid recommender than to just create it.
The idea of this project was to model and create a hybrid recommender that takes **multiple factors** into account, **NOT** to actually make a relevant movie recommender. As this was **never** possible in the first place. The simple reason for this being that the data sources don't share any movies in common. Nonetheless during the process of creating this we cleaned and transformed the data **AS IF** it was possible, **AS IF** they shared movies in common. eg. by using IMDb id for the movies in both datasets, using identical format for title values, same output format and etc. </font>

In [ ]:
#Commonly shared movies
df1[['movie_id', 'title']].drop_duplicates().append(df2[['movie_id', 'title']].drop_duplicates()).value_counts().sort_values(ascending=False)


# Project Reflection
<font size=3>With only some python experience and no practical experience in machine learning, I spent the major portion of my time reading about different machine learning algorithms, how to use them, how recommendation systems are created, etc before actually starting to code this movie recommender. Therefore there was only so much I could do with the time. I'm however seriously proud of myself and this project because what's not showcased here is the journey of how much I've learned and developed in this field. </font>

<font size=4>**What would I have done differently?**</font>

* <font size=3>To keep a relevancy between the recommender systems, I should've probably stuck to same algorithm or normalized the similarity results by eachother.</font>
* <font size=3>I should have analyzed the accuracy of the algorithms as well as test the different feature combinations in the content based recommender to see what data creates most accuracte predictions (Hard to measure this though, would probably need to create model from a manually-curated list of movies from the data that are similar to eachother and compare the results between the models on those movies)</font>
* <font size=3>Used data visualizations to analyse and showcase the data more in-depth and clear. </font>
* <font size=3>Retrieving more accurate keywords by filtering other useless words rather than just stopwords from description (Research more NLP or using rake?)</font>


